# 1. Import Libraries:

In [1]:
base_dir = "/kaggle/input/tomatoleaf/tomato"
train_dir = "/kaggle/input/tomatoleaf/tomato/train"
validation_dir = "/kaggle/input/tomatoleaf/tomato/val"

# 2. Define Image Size and Batch Size:

In [2]:
IMAGE_SIZE = 224 # define the image size for all images (height and Width = 224 X 224)
BATCH_SIZE = 64  # At the time of training 64 images will be there at a time

# 3. Set Folder Paths:

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from glob import glob
from sklearn.model_selection import train_test_split

# 4. Get the Number of Classes from Folder Names:

In [4]:
folders = glob(train_dir + '/*')
print(len(folders))

num_classes = len(folders)
print("Number of classes:", num_classes)

10
Number of classes: 10


# 5. Load VGG16 Model:

In [5]:
# IMAGE_SIZE = [224, 224]
vgg_model = VGG16(weights = 'imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)) 
# include_top = False (where we are defining our own class/label) means we are working at our own dataset where we will have our own label
# here we will have 10 layers (as 10 classes are there)
vgg_model.output

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


<KerasTensor shape=(None, 7, 7, 512), dtype=float32, sparse=False, name=keras_tensor_18>

# 6. Modify the output layer

In [6]:
x = Flatten()(vgg_model.output)
# Define additional dense layers
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout regularization to prevent overfitting
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)  # Add dropout regularization
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)  # Add dropout regularization
# Dense layer will add neurons of the lenth of folders 
prediction = Dense(len(folders), activation="softmax")(x) # here x is the output of vgg

# 7. Freeze Convolutional Layers (Optional Fine-Tuning):

In [7]:
# unfreeze layers
for layer in vgg_model.layers[-10:]:
    layer.trainable = True
# We have total 19 layers

# 8. Create a new model with the new output layer

In [8]:
model = Model(inputs=vgg_model.input, outputs=prediction)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │    25,691,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,32

 Total params: 41,064,522 (156.65 MB)

 Trainable params: 41,064,522 (156.65 MB)

 Non-trainable params: 0 (0.00 B)

# 8. Compile the Model (Initial Training):

In [9]:
rms_optimizer = RMSprop(learning_rate=0.0001, rho=0.9)  # Consider using a learning rate scheduler
model.compile(loss="categorical_crossentropy", optimizer=rms_optimizer, metrics=["accuracy"])

## Prepare the data.

# 9. Data Augmentation for Training and Validation:

In [10]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split = 0.2  # 20% for validation
)
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split = 0.2 # 20% for validation
)
test_datagen =  ImageDataGenerator(
    rescale=1./255,
)

# 10. Prepare Data Generators:

In [11]:
train_generator = train_datagen.flow_from_directory(
    train_dir, # define directory
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,  # Shuffle the data before splitting
    subset='training',  # Specify that this is for training
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',  # Specify that this is for training
)
test_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    shuffle = False,
    class_mode = 'categorical'
)

Found 8000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


# 11. Early Stopping and Learning Rate Reduction:

In [14]:
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, min_lr=0.0001)
model_checkpoint = ModelCheckpoint('tomato_vgg16_model_update.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=1)

# 12. Train the Model (Initial):

In [15]:
epoch = 20
history = model.fit(
    train_generator, epochs=epoch, validation_data=validation_generator, callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 975ms/step - accuracy: 0.1012 - loss: 2.3437

W0000 00:00:1715768021.694370      83 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
2024-05-15 10:14:12.154299: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 3.89842, expected 3.37692
2024-05-15 10:14:12.154357: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 12: 3.13412, expected 2.61262
2024-05-15 10:14:12.154367: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 13: 3.08377, expected 2.56227
2024-05-15 10:14:12.154374: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 14: 4.1021, expected 3.5806
2024-05-15 10:14:12.154382: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 29: 3.76539, expected 3.24388
2024-05-15 10:14:12.154390: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 30: 4.19841, expected 3.6769
2024-05-15 10:14:12.154398: E external/local_xla/xla/service/gp


Epoch 1: val_loss improved from inf to 2.17929, saving model to tomato_vgg16_model_update.weights.h5
125/125 ━━━━━━━━━━━━━━━━━━━━ 173s 1s/step - accuracy: 0.1012 - loss: 2.3435 - val_accuracy: 0.1600 - val_loss: 2.1793 - learning_rate: 1.0000e-04
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 745ms/step - accuracy: 0.1613 - loss: 2.2061
Epoch 2: val_loss improved from 2.17929 to 2.14443, saving model to tomato_vgg16_model_update.weights.h5
125/125 ━━━━━━━━━━━━━━━━━━━━ 123s 941ms/step - accuracy: 0.1616 - loss: 2.2055 - val_accuracy: 0.1660 - val_loss: 2.1444 - learning_rate: 1.0000e-04
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 741ms/step - accuracy: 0.2902 - loss: 1.9036
Epoch 3: val_loss improved from 2.14443 to 1.53859, saving model to tomato_vgg16_model_update.weights.h5
125/125 ━━━━━━━━━━━━━━━━━━━━ 122s 936ms/step - accuracy: 0.2906 - loss: 1.9029 - val_accuracy: 0.4595 - val_loss: 1.5386 - learning_rate: 1.0000e-04
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 747ms/step - accuracy: 0.

# 13. Evaluate the Model after Initial Training:

In [17]:
score = model.evaluate(test_generator)
print("Test loss (initial):", score[0])
print("Test accuracy (initial):", score[1])

16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 152ms/step - accuracy: 0.9254 - loss: 0.2087
Test loss (initial): 0.2161996215581894
Test accuracy (initial): 0.925000011920929


# Predict

# 14. Get Classification Report

In [18]:
import numpy as np
from sklearn.metrics import classification_report
class_labels = ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 
                'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite',
                'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus',
                'Tomato___healthy']

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_labels = test_generator.classes
report = classification_report(true_labels, predicted_classes, target_names=class_labels)
print(report)

 2/16 ━━━━━━━━━━━━━━━━━━━━ 2s 148ms/step 

W0000 00:00:1715770084.629282      84 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 186ms/step
                                               precision    recall  f1-score   support

                      Tomato___Bacterial_spot       0.95      0.91      0.93       100
                        Tomato___Early_blight       0.86      0.96      0.91       100
                         Tomato___Late_blight       0.95      0.93      0.94       100
                           Tomato___Leaf_Mold       1.00      0.96      0.98       100
                  Tomato___Septoria_leaf_spot       0.93      0.90      0.91       100
Tomato___Spider_mites Two-spotted_spider_mite       0.96      0.91      0.93       100
                         Tomato___Target_Spot       0.82      0.82      0.82       100
       Tomato___Tomato_Yellow_Leaf_Curl_Virus       0.98      0.97      0.97       100
                 Tomato___Tomato_mosaic_virus       0.99      0.90      0.94       100
                             Tomato___healthy       0.85      0.99      0.91       100



W0000 00:00:1715770087.417508      81 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


# 15. Update the weights of the model

In [19]:
model.load_weights('/kaggle/working/tomato_vgg16_model_update.weights.h5')

# Test an image for prediction

In [20]:
import numpy as np
import os
from keras.preprocessing import image
from keras.models import load_model

image_path = "/kaggle/input/tomatoleaf/tomato/val/Tomato___Septoria_leaf_spot/0a25f893-1b5f-4845-baa1-f68ac03d96ac___Matt.S_CG 7863.JPG"

# Extract the actual class label from the image path
actual_class = os.path.basename(os.path.dirname(image_path))

img_pred = image.load_img(image_path, target_size=(224,224))
img_pred = image.img_to_array(img_pred)
img_pred = np.expand_dims(img_pred, axis=0) # adding a dimension

# Define the list of class labels
class_labels = ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 
                'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite',
                'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus',
                'Tomato___healthy']
# passing the array to the model 
# vgg_model = load_model('/kaggle/working/vgg_model_tomato.h5')
# reslt = vgg_model.predict(img_pred) # return a 2d array
reslt = model.predict(img_pred)
print(reslt)

# Get the predicted class index
predicted_class_index = np.argmax(reslt[0])

# Print the predicted class
predicted_class = class_labels[predicted_class_index]
print("Predicted class:", predicted_class)

# Print the actual class
print("Actual class:", actual_class)

2024-05-15 10:49:18.089014: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 4.42855, expected 3.64542
2024-05-15 10:49:18.089066: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 5.87036, expected 5.08723
2024-05-15 10:49:18.089076: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 6.04003, expected 5.2569
2024-05-15 10:49:18.089084: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 6.22034, expected 5.43722
2024-05-15 10:49:18.089091: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 4.90026, expected 4.11714
2024-05-15 10:49:18.089099: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 5.21068, expected 4.42756
2024-05-15 10:49:18.089106: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 5.92061, expected 5.13748
2024-05-15 10:49:18.089114: E external/local_xla/xla/ser

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Predicted class: Tomato___Bacterial_spot
Actual class: Tomato___Septoria_leaf_spot


# Visualize the Output

## 1. Evaluating the Model